In [9]:
import numpy as np
import pandas as pd

# importing the requests library
import requests

In [10]:
## Resources ##
# [1] https://www.arcgis.com/home/item.html?id=124c2187da8c41c59bde04fa67eb2872
# [2] https://data-wake.opendata.arcgis.com/datasets/Wake::restaurants-in-wake-county/api

In [11]:
## Sourcing Data ##

def getRestaurantsDf():
        # Source restaurants data in wake county from wakegov
        base_url = "https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/0/query?f=json&where=(FACILITYTYPE%20IN%20('Restaurant'))&outFields=*"

        # keep pagination in mind (maxRecordCount = 1000, so grabbing 1000 at a time)
        reqs = []
        reqs.append(base_url)
        reqs.append(base_url+'&resultOffset=1000&resultRecordCount=1000')
        reqs.append(base_url+'&resultOffset=2000&resultRecordCount=1000')
        reqs.append(base_url+'&resultOffset=3000&resultRecordCount=1000')

        # Sending get request and saving the response as response object
        # extracting data in json 
        rows = []
        for val in reqs:
            r = requests.get(url = val)
            data = r.json()['features']
            for d in data:
                rows.append(d['attributes'])
        return pd.DataFrame(rows)


restaurants_df = getRestaurantsDf()

In [12]:
# Validate shape & data
# https://data-wake.opendata.arcgis.com/datasets/Wake::restaurants-in-wake-county/explore?filters=eyJGQUNJTElUWVRZUEUiOlsiUmVzdGF1cmFudCJdfQ%3D%3D&location=35.794181%2C-78.605600%2C3.98&showTable=true
print(restaurants_df.shape)
print(restaurants_df['OBJECTID'].nunique())
print(restaurants_df['HSISID'].nunique())
display(restaurants_df.head())

(2385, 15)
1987
1987


,OBJECTID,HSISID,NAME,ADDRESS1,ADDRESS2,CITY,STATE,POSTALCODE,PHONENUMBER,RESTAURANTOPENDATE,FACILITYTYPE,PERMITID,X,Y,GEOCODESTATUS
0,1853060,04092018384,Curry House,2101 S MAIN ST,None,WAKE FOREST,NC,27587,(919) 985-7244,1592352000000,Restaurant,16526,-78.533101,35.948021,M
1,1853061,04092018571,Wendy's,7132 US Highway 64 E,None,KNIGHTDALE,NC,27545,(919) 266-0917,1616544000000,Restaurant,16463,0.000000,0.000000,U
2,1853062,04092018664,Akami Sushi Bar,1561 E WILLIAMS ST,None,APEX,NC,27539,(919) 267-6368,1624924800000,Restaurant,16528,-78.835346,35.706758,M
3,1853063,04092018340,Seoul Garden,4701-131 Atlantic Ave,None,RALEIGH,NC,27604,(919) 850-9984,1586217600000,Restaurant,16468,-78.601777,35.848223,M
4,1853065,04092015374,Shane's Rib Shack #55,5811 POYNER VILLAGE PKY,SUITE 101,RALEIGH,NC,27616,(919) 850-9900,1239753600000,Restaurant,16546,-78.569166,35.864643,M


In [24]:
# https://data-wake.opendata.arcgis.com/datasets/food-inspections/explore
# https://www.arcgis.com/home/item.html?id=ebe3ae7f76954fad81411612d7c4fb17
def getFoodInspectionsDf(): 
    # https://maps.wakegov.com/arcgis/rest/services/Inspections/RestaurantInspectionsOpenData/MapServer/1/query?outFields=*&where=1%3D1
    
    # Sending get request and saving the response as response object
    # extracting data in json
    val = 'https://opendata.arcgis.com/datasets/ebe3ae7f76954fad81411612d7c4fb17_1.geojson'
    rows = []
    r = requests.get(url = val)
    data = r.json()
    #print(data)
    for d in data:
        data = r.json()['features']
        for d in data:
            rows.append(d['properties'])
    return pd.DataFrame(rows)

food_inspections_df = getFoodInspectionsDf()

In [28]:
print(food_inspections_df.shape)
food_inspections_df.head()

(189396, 8)


,OBJECTID,HSISID,SCORE,DATE_,DESCRIPTION,TYPE,INSPECTOR,PERMITID
0,21427819,04092017542,94.5,2017-04-07T00:00:00Z,"Inspection conducted by Joshua Volkan, supervi...",Inspection,Anne-Kathrin Bartoli,108
1,21427820,04092017542,92.0,2017-11-08T00:00:00Z,manager owns two deep chest freezers that are ...,Inspection,Laura McNeill,108
2,21427821,04092017542,95.0,2018-03-23T00:00:00Z,None,Inspection,Laura McNeill,108
3,21427822,04092017542,93.5,2018-09-07T00:00:00Z,"*NOTICE* EFFECTIVE JANUARY 1, 2019, THE NC FOO...",Inspection,Laura McNeill,108
4,21427823,04092017542,93.0,2019-04-04T00:00:00Z,"*NOTICE* AS OF JANUARY 1, 2019, THE NC FOOD CO...",Inspection,Joanne Rutkofske,108
